In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import time 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
movies_df = pd.read_csv('/content/drive/MyDrive/Projects/RBM-main/RBM_Recommendation/ml-1m/movies.dat', sep='::', header=None, engine='python',encoding='latin-1')
print(movies_df.head())

   0                                   1                             2
0  1                    Toy Story (1995)   Animation|Children's|Comedy
1  2                      Jumanji (1995)  Adventure|Children's|Fantasy
2  3             Grumpier Old Men (1995)                Comedy|Romance
3  4            Waiting to Exhale (1995)                  Comedy|Drama
4  5  Father of the Bride Part II (1995)                        Comedy


In [ ]:
ratings_df = pd.read_csv('/content/drive/MyDrive/Projects/RBM-main/RBM_Recommendation/ml-1m/ratings.dat', sep='::', header=None, engine='python',encoding='latin-1')
print(ratings_df.head())

   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968
3  1  3408  4  978300275
4  1  2355  5  978824291


In [ ]:
movies_df.columns = ['MovieID', 'Title', 'Genres']
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']

# Verify the changes done to the dataframes
print(movies_df.head())
print(ratings_df.head())

   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


In [ ]:
n_users = ratings_df.UserID.unique().shape[0]
n_movies = ratings_df.MovieID.unique().shape[0]
n_ratings = len(ratings_df)

avg_ratings_per_user = n_ratings/n_users

print('Number of unique users: ', n_users)
print('Number of unique movies: ', n_movies)
print('Number of total ratings: ', n_ratings)
print('Average number of ratings per user: ', avg_ratings_per_user)

Number of unique users:  6040
Number of unique movies:  3706
Number of total ratings:  1000209
Average number of ratings per user:  165.5975165562914


In [ ]:
# Data Correction and Formatting
print('The Number of Movies in Dataset', len(movies_df))

"""
- Our Movie ID's vary from 1 to 3952 while we have 3883 movies. 
- Due to this, we won't be able to index movies through their ID since we would get memory indexing errors. 
- To amend we can create a column that shows the spot in our list that particular movie is in:
"""

movies_df['List Index'] = movies_df.index
print(movies_df.head())

# Merge movies_df with ratings_df by MovieID
merged_df = movies_df.merge(ratings_df, on='MovieID')

# Drop unnecessary columns
merged_df = merged_df.drop('Timestamp', axis=1).drop('Title', axis=1).drop('Genres', axis=1)

# Display the result
print(merged_df.head())

The Number of Movies in Dataset 3883
   MovieID  ... List Index
0        1  ...          0
1        2  ...          1
2        3  ...          2
3        4  ...          3
4        5  ...          4

[5 rows x 4 columns]
   MovieID  List Index  UserID  Rating
0        1           0       1       5
1        1           0       6       4
2        1           0       8       4
3        1           0       9       5
4        1           0      10       5


In [ ]:
user_Group = merged_df.groupby('UserID')
print(user_Group.head())

        MovieID  List Index  UserID  Rating
0             1           0       1       5
1             1           0       6       4
2             1           0       8       4
3             1           0       9       5
4             1           0      10       5
...         ...         ...     ...     ...
874890     3285        3216    1270       1
883271     3357        3288    4896       5
903916     3448        3379    4025       5
928813     3548        3479     986       3
933205     3568        3499    1226       3

[30200 rows x 4 columns]


In [ ]:
# Amount of users used for training
start_time=time.time()

ratings_df = ratings_df.sample(frac=1, random_state=42)
x = ratings_df[["UserID", "MovieID"]].values
print(x)
# Normalize the targets between 0 and 1. Makes it easy to train.
y = user_Group["Rating"].apply(lambda x:x/5.0).values
print(y)
# Assuming training on 80% of the data and validating on 10%.
train_indices = int(0.8 * ratings_df.shape[0])
X_train, X_val, Y_train, Y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

print('Size of train set: ', len(X_train))
print('Size of test set: ', len(X_val))





#print(trX)
print(time.time()-start_time)

[[5412 2683]
 [5440  904]
 [ 368 3717]
 ...
 [ 854 3102]
 [4033 3479]
 [ 786 1391]]
[1.  0.8 0.8 ... 0.8 0.2 0.8]
Size of train set:  800167
Size of test set:  200042
1.666654109954834


In [ ]:
# Setting the models Parameters
with tf.device("/device:GPU:0"):
    start_time=time.time()
    hiddenUnits = 50
    visibleUnits = len(movies_df)
    vb = tf.placeholder(tf.float32, [visibleUnits])  # Number of unique movies
    hb = tf.placeholder(tf.float32, [hiddenUnits])  # Number of features were going to learn
    W = tf.placeholder(tf.float32, [visibleUnits, hiddenUnits])  # Weight Matrix

# Phase 1: Input Processing
    v0 = tf.placeholder("float", [None, visibleUnits])
    _h0 = tf.nn.sigmoid(tf.matmul(v0, W) + hb)  # Visible layer activation
    h0 = tf.nn.relu(tf.sign(_h0 - tf.random_uniform(tf.shape(_h0))))  # Gibb's Sampling

# Phase 2: Reconstruction
    _v1 = tf.nn.sigmoid(tf.matmul(h0, tf.transpose(W)) + vb)  # Hidden layer activation
    v1 = tf.nn.relu(tf.sign(_v1 - tf.random_uniform(tf.shape(_v1))))
    h1 = tf.nn.sigmoid(tf.matmul(v1, W) + hb)

""" Set RBM Training Parameters """

# Learning rate
with tf.device("/device:GPU:0"):
    alpha = 1.0

# Create the gradients
    w_pos_grad = tf.matmul(tf.transpose(v0), h0)
    w_neg_grad = tf.matmul(tf.transpose(v1), h1)

# Calculate the Contrastive Divergence to maximize
    CD = (w_pos_grad - w_neg_grad) / tf.to_float(tf.shape(v0)[0])

# Create methods to update the weights and biases
    update_w = W + alpha * CD
    update_vb = vb + alpha * tf.reduce_mean(v0 - v1, 0)
    update_hb = hb + alpha * tf.reduce_mean(h0 - h1, 0)

# Set the error function, here we use Mean Absolute Error Function
    err = v0 - v1
    err_sum = tf.reduce_mean(err*err)

""" Initialize our Variables with Zeroes using Numpy Library """
with tf.device("/device:GPU:0"):
# Current weight
    cur_w = np.zeros([visibleUnits, hiddenUnits], np.float32)

# Current visible unit biases
    cur_vb = np.zeros([visibleUnits], np.float32)

# Current hidden unit biases
    cur_hb = np.zeros([hiddenUnits], np.float32)

# Previous weight
    prv_w = np.zeros([visibleUnits, hiddenUnits], np.float32)

# Previous visible unit biases
    prv_vb = np.zeros([visibleUnits], np.float32)

# Previous hidden unit biases
    prv_hb = np.zeros([hiddenUnits], np.float32)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

# Train RBM with 15 Epochs, with Each Epoch using 10 batches with size 100, After training print out the error by epoch
    with tf.device("/device:GPU:0"):    
        epochs = 100
        batchsize = 300
        errors = []
        for i in range(epochs):
            for start, end in zip(range(0, len(X_train), batchsize), range(batchsize, len(X_train), batchsize)):
                batch = X_train[start:end]
                cur_w = sess.run(update_w, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
                cur_vb = sess.run(update_vb, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
                cur_hb = sess.run(update_hb, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
                prv_w = cur_w
                prv_vb = cur_vb
                prv_hb = cur_hb
            errors.append(sess.run(err_sum, feed_dict={v0: X_train, W: cur_w, vb: cur_vb, hb: cur_hb}))
            print(errors[-1])
    
        print(time.time()-start_time)
        plt.plot(errors)
        plt.ylabel('Error')
        plt.xlabel('Epoch')
        plt.show()

InvalidArgumentError: ignored

In [ ]:
"""
Recommendation System :-
- We can now predict movies that an arbitrarily selected user might like. 
- This can be accomplished by feeding in the user's watched movie preferences into the RBM and then reconstructing the 
  input. 
- The values that the RBM gives us will attempt to estimate the user's preferences for movies that he hasn't watched 
  based on the preferences of the users that the RBM was trained on.
"""

# Select the input User
user=202
inputUser = [X_test[user]]

# Feeding in the User and Reconstructing the input
hh0 = tf.nn.sigmoid(tf.matmul(v0, W) + hb)
vv1 = tf.nn.sigmoid(tf.matmul(hh0, tf.transpose(W)) + vb)
feed = sess.run(hh0, feed_dict={v0: inputUser, W: prv_w, hb: prv_hb})
rec = sess.run(vv1, feed_dict={hh0: feed, W: prv_w, vb: prv_vb})

# List the 20 most recommended movies for our mock user by sorting it by their scores given by our model.
scored_movies_df_input = movies_df
scored_movies_df_input["Recommendation Score"] = rec[0]
print(scored_movies_df_input.sort_values(["Recommendation Score"], ascending=False).head(20))



NameError: ignored

In [ ]:
""" Recommend User what movies he has not watched yet """

# Find the mock user's UserID from the data
mock_userid= merged_df.iloc[user]
print(mock_userid)

# Find all movies the mock user has watched before
movies_df_input = merged_df[merged_df['UserID'] == (mock_userid.UserID)]
print(movies_df_input.head())



In [ ]:
""" Merge all movies that our mock users has watched with predicted scores based on his historical data: """

# Merging movies_df with ratings_df by MovieID
merged_df_input = scored_movies_df_input.merge(movies_df_input, on='MovieID', how='outer')

# Dropping unnecessary columns
merged_df_input = merged_df_input.drop('List Index_y', axis=1).drop('UserID', axis=1)

# Sort and take a look at first 20 rows
print(merged_df_input.sort_values(['Recommendation Score'], ascending=False).head(20))

In [ ]:
f = open('results.csv', "w+")
f.close()

In [ ]:
sorted_result = merged_df_input.sort_values(by='Recommendation Score', ascending=False)

In [ ]:
sorted_result.to_csv(r'D:\ACADEMICS\RECOMMENDATION ENGINE\results.csv')